In [1]:
import cobra
from cobra.io import load_json_model
from cobra import Model, Reaction, Metabolite
from cobra.flux_analysis import flux_variability_analysis

import pandas as pd
import numpy as np

import escher
from escher import Builder

# 1. Define medium

Synthetic medium from https://www.sciencedirect.com/science/article/pii/S0014579307007235
<br>"minimal synthetic medium containing 48 mM Na2HPO4, 22 mM KH2PO4, 9 mM NaCl, 19 mM NH4Cl, 2 mM MgSO4, 0.1 mM CaCl2, 0.1 g/l of thiamine and 2 g/l of glucose"

In [35]:
# Load the model
EcN_model = cobra.io.load_json_model('../data/models/CP022686.1_cur_4.8.json')

In [36]:
# Define the biolog media
medium = EcN_model.medium
medium['EX_glc__D_e'] = 15
    
medium['EX_nh4_e'] = 19 # 19 mM NH4Cl
medium['EX_pi_e'] = 60 # 48mM Na2HPO4 + 22 mM KH2PO4  
medium['EX_so4_e'] = 2 # 2 mM MgSO4
    
medium['EX_mg2_e'] = 2 # 2 mM MgSO4
medium['EX_ca2_e'] = 0.1 # 0.1 mM CaCl2SS
    
medium['EX_na1_e'] = 105 # 48 mM Na2HPO4 + 9 mM NaCl
medium['EX_k_e'] = 22 # 22 mM KH2PO4
medium['EX_cl_e'] = 28 # 9 mM NaCl + 19 mM NH4Cl
    
medium['EX_thm_e'] = 0.38 # 0.1 g/L > 0.1/265.355 x 1000 = 0.376854 mM

# trace elements were not defined in the paper
value = 1000 # 0.01
medium['EX_fe3_e'] = value
medium['EX_mn2_e'] = value
medium['EX_fe2_e'] = value
medium['EX_zn2_e'] = value
medium['EX_ni2_e'] = value
medium['EX_cu2_e'] = value
medium['EX_cobalt2_e'] = value
medium['EX_sel_e'] = value
medium['EX_mobd_e'] = value
medium['EX_tungs_e'] = value
medium['EX_slnt_e'] = value
medium['EX_salchsx_e'] = value

EcN_model.medium = medium

In [37]:
# Adjust the flow through several reactions based on Kim et al. 2021
for rxn in ['CELBpts', 'CLBtex', 'HPAtex', '3hoxpactex', 'LCARS', 'PTRCORNt7pp', 'DTARTD', 'SUCTARTtpp', 'TARTD', 'TARTRt7pp']: # CELBpts describes the cellobiose reaction 
    EcN_model.reactions.get_by_id(rxn).lower_bound = 0
    EcN_model.reactions.get_by_id(rxn).upper_bound = 0

# 2. FVA

In [38]:
# # Set acetate flux > 0.481 ± 0.001 * 15 > 
# # EcN_model.reactions.EX_ac_e.bounds = (0.480*15, 0.482*15)
# EcN_model.reactions.ACKr.bounds = (0.480*15, 0.482*15)

In [39]:
# wt_solution = EcN_model.optimize()
# wt_growth = wt_solution.fluxes["BIOMASS_EcN_iHM1533_core_59p80M"]
# acetate_production = wt_solution.fluxes["EX_ac_e"]

In [40]:
# #set growth to 99% of maximum for FVA
# FVA_value = 0.90
# model_copy = EcN_model.copy()

# # Set FVA value as growth boundaries
# growth = FVA_value*model_copy.optimize().objective_value
# model_copy.reactions.BIOMASS_EcN_iHM1533_core_59p80M.bounds = (growth,growth)

# print(growth)

### Import flux data

In [41]:
# # Import the flux file and set index
# flux_df = pd.read_excel('../tables/13C_fluxes.xlsx', usecols=(0,1,2,3))
# flux_df = flux_df.dropna()
# flux_df.set_index('ID', inplace=True)
# flux_df.sort_values(by='glc_flux_mean', ascending=True, inplace=True)
# flux_df['Flux_x'] = flux_df['glc_flux_mean'] *15
# flux_df['LB'] = (flux_df['glc_flux_mean'] - flux_df['glc_flux_sd']) *15 # normalise for 15 glucose
# flux_df['UB'] = (flux_df['glc_flux_mean'] + flux_df['glc_flux_sd']) *15 # normalise for 15 glucose
# flux_df.head()

In [42]:
# for index, row in flux_df.iterrows():  
#     bounds = flux_variability_analysis(model_copy, index)

#     # add model_id column
#     flux_df.loc[index,'model_id'] = index
    
#     # Add in silico prediction
#     min_b = bounds['minimum'][0]
#     max_b = bounds['maximum'][0]
#     flux_df.loc[index,'silico_LB'] = min_b#/15
#     flux_df.loc[index,'silico_UB'] = max_b#/15
#     flux_df.loc[index,'silico_LB_abs'] = abs(min_b)#/15
#     flux_df.loc[index,'silico_UB_abs'] = abs(max_b)#/15
    
#     # Add absolute flux
#     flux_df.loc[index,'flux_abs'] = abs(flux_df.loc[index,'glc_flux_mean'])

In [43]:
# flux_df

In [44]:
# # The values of PGK are negative, as the reaction is defined opposite
# flux_df.loc['PGK', 'silico_LB'] = -flux_df.loc['PGK', 'silico_LB']
# flux_df.loc['PGK', 'silico_UB'] = -flux_df.loc['PGK', 'silico_UB']

# # Save dataframe
# flux_df.to_csv('../tables/Outputs/flux_pred_%s.csv'%FVA_value)

### Combine all in one

In [45]:
### Set model specifications ###

# Set acetate flux > 0.481 ± 0.001 * 15 > Model describes flux from AcCoA to Acetate, use ACKr
# EcN_model.reactions.EX_ac_e.bounds = (0.480*15, 0.482*15)
EcN_model.reactions.ACKr.bounds = (0.480*15, 0.482*15)



### Set FVA values ### 
FVA_range = [0.9, 0.95, 0.96, 0.97, 0.98, 0.99]    
for FVA_value in FVA_range:  
    #set growth to 99% of maximum for FVA
#     FVA_value = 0.90
    model_copy = EcN_model.copy()

    # Set FVA value as growth boundaries
    growth = FVA_value*model_copy.optimize().objective_value
    model_copy.reactions.BIOMASS_EcN_iHM1533_core_59p80M.bounds = (growth,growth)

    print(growth)
    
    
    
### Import flux data ###

    # Import the flux file and set index
    flux_df = pd.read_excel('../tables/13C_fluxes.xlsx', usecols=(0,1,2,3,7))
    flux_df = flux_df.dropna()
    flux_df.set_index('ID', inplace=True)
    flux_df.sort_values(by='number', ascending=True, inplace=True)
    flux_df['Flux_x'] = flux_df['glc_flux_mean'] *15
    flux_df['LB'] = (flux_df['glc_flux_mean'] - flux_df['glc_flux_sd']) *15 # normalise for 15 glucose
    flux_df['UB'] = (flux_df['glc_flux_mean'] + flux_df['glc_flux_sd']) *15 # normalise for 15 glucose

    for index, row in flux_df.iterrows():  
        bounds = flux_variability_analysis(model_copy, index)

        # add model_id column
        flux_df.loc[index,'model_id'] = index

        # Add in silico prediction
        min_b = bounds['minimum'][0]
        max_b = bounds['maximum'][0]
        flux_df.loc[index,'silico_LB'] = min_b#/15
        flux_df.loc[index,'silico_UB'] = max_b#/15
        flux_df.loc[index,'silico_LB_abs'] = abs(min_b)#/15
        flux_df.loc[index,'silico_UB_abs'] = abs(max_b)#/15

        # Add absolute flux
        flux_df.loc[index,'flux_abs'] = abs(flux_df.loc[index,'glc_flux_mean'])

        
### Correct dataframe and save ###
    # The values of PGK are negative, as the reaction is defined opposite
    flux_df.loc['PGK', 'silico_LB'] = -flux_df.loc['PGK', 'silico_LB']
    flux_df.loc['PGK', 'silico_UB'] = -flux_df.loc['PGK', 'silico_UB']
    flux_df.drop(('ACKr'), inplace=True)


    # Save dataframe
    flux_df.to_csv('../tables/Outputs/flux_pred_%s.csv'%FVA_value)

0.7997902997885831
0.844223094221282
0.8531096531078218
0.8619962119943617
0.8708827708809015
0.8797693297674414


# pFBA

In [46]:
model_copy = EcN_model.copy()

# Set acetate flux > 0.481 ± 0.001 * 15 > 
# EcN_model.reactions.EX_ac_e.bounds = (0.480*15, 0.482*15)
EcN_model.reactions.ACKr.bounds = (0.480*15, 0.482*15)

In [47]:
pfba_solution = cobra.flux_analysis.pfba(model_copy)

In [48]:
#set the pfba_factor to 1.01 to find variability of pfba 
#i.e. this corresponds to 99% FVA
pfba_solution_101 = flux_variability_analysis(model_copy,pfba_factor = 1.01)

In [49]:
#make a dataframe & export
for index, row in flux_df.iterrows():  
    pfba = pfba_solution[row['model_id']]
    mini = pfba_solution_101.loc[row['model_id']]['minimum']
    maxi = pfba_solution_101.loc[row['model_id']]['maximum']
    flux_df.loc[index,'pfba'] = pfba
    flux_df.loc[index,'pfba_101_min'] = mini
    flux_df.loc[index,'pfba_101_max'] = maxi

In [50]:
flux_df.to_csv('../tables/Outputs/flux_pred_pfba.csv')

# Gluconate

In [18]:
# Load the model
EcN_model = cobra.io.load_json_model('../data/models/CP022686.1_cur_4.8.json')

# Define the biolog media
medium = EcN_model.medium
medium['EX_glc__D_e'] = 0
medium['EX_glcn_e'] = 15
    
medium['EX_nh4_e'] = 19 # 19 mM NH4Cl
medium['EX_pi_e'] = 60 # 48mM Na2HPO4 + 22 mM KH2PO4  
medium['EX_so4_e'] = 2 # 2 mM MgSO4
    
medium['EX_mg2_e'] = 2 # 2 mM MgSO4
medium['EX_ca2_e'] = 0.1 # 0.1 mM CaCl2SS
    
medium['EX_na1_e'] = 105 # 48 mM Na2HPO4 + 9 mM NaCl
medium['EX_k_e'] = 22 # 22 mM KH2PO4
medium['EX_cl_e'] = 28 # 9 mM NaCl + 19 mM NH4Cl
    
medium['EX_thm_e'] = 0.38 # 0.1 g/L > 0.1/265.355 x 1000 = 0.376854 mM

# trace elements were not defined in the paper
value = 1000 # 0.01
medium['EX_fe3_e'] = value
medium['EX_mn2_e'] = value
medium['EX_fe2_e'] = value
medium['EX_zn2_e'] = value
medium['EX_ni2_e'] = value
medium['EX_cu2_e'] = value
medium['EX_cobalt2_e'] = value
medium['EX_sel_e'] = value
medium['EX_mobd_e'] = value
medium['EX_tungs_e'] = value
medium['EX_slnt_e'] = value
medium['EX_salchsx_e'] = value

EcN_model.medium = medium

### FVA

In [19]:
### Set model specifications ###

# Set acetate flux > 0.481 ± 0.001 * 15 > Model describes flux from AcCoA to Acetate, use ACKr
# EcN_model.reactions.EX_ac_e.bounds = (0.480*15, 0.482*15)
EcN_model.reactions.ACKr.bounds = (0.480*15, 0.482*15)



### Set FVA values ### 
FVA_range = [0.9, 0.95, 0.96, 0.97, 0.98, 0.99]    
for FVA_value in FVA_range:  
    #set growth to 99% of maximum for FVA
#     FVA_value = 0.90
    model_copy = EcN_model.copy()

    # Set FVA value as growth boundaries
    growth = FVA_value*model_copy.optimize().objective_value
    model_copy.reactions.BIOMASS_EcN_iHM1533_core_59p80M.bounds = (growth,growth)

    print(growth)
    
    
    
### Import flux data ###

    # Import the flux file and set index
    flux_df = pd.read_excel('../tables/13C_fluxes.xlsx', usecols=(0,1,4,5,7))
    flux_df = flux_df.dropna()
    flux_df.set_index('ID', inplace=True)
    flux_df.sort_values(by='number', ascending=True, inplace=True)
    flux_df['Flux_x'] = flux_df['glcn_flux_mean'] *15
    flux_df['LB'] = (flux_df['glcn_flux_mean'] - flux_df['glcn_flux_sd']) *15 # normalise for 15 glucose
    flux_df['UB'] = (flux_df['glcn_flux_mean'] + flux_df['glcn_flux_sd']) *15 # normalise for 15 glucose

    for index, row in flux_df.iterrows():  
        bounds = flux_variability_analysis(model_copy, index)

        # add model_id column
        flux_df.loc[index,'model_id'] = index

        # Add in silico prediction
        min_b = bounds['minimum'][0]
        max_b = bounds['maximum'][0]
        flux_df.loc[index,'silico_LB'] = min_b#/15
        flux_df.loc[index,'silico_UB'] = max_b#/15
        flux_df.loc[index,'silico_LB_abs'] = abs(min_b)#/15
        flux_df.loc[index,'silico_UB_abs'] = abs(max_b)#/15

        # Add absolute flux
        flux_df.loc[index,'flux_abs'] = abs(flux_df.loc[index,'glcn_flux_mean'])

        
### Correct dataframe and save ###
    # The values of PGK are negative, as the reaction is defined opposite
    flux_df.loc['PGK', 'silico_LB'] = -flux_df.loc['PGK', 'silico_LB']
    flux_df.loc['PGK', 'silico_UB'] = -flux_df.loc['PGK', 'silico_UB']
    flux_df.drop(('ACKr'), inplace=True)


    # Save dataframe
    flux_df.to_csv('../tables/Outputs/glcn_flux_pred_%s.csv'%FVA_value)

0.7484378102180383
0.7900176885634849
0.7983336642325742
0.8066496399016635
0.8149656155707529
0.8232815912398421


### pFBA

In [20]:
model_copy = EcN_model.copy()

# Set acetate flux > 0.481 ± 0.001 * 15 > 
# EcN_model.reactions.EX_ac_e.bounds = (0.480*15, 0.482*15)
EcN_model.reactions.ACKr.bounds = (0.480*15, 0.482*15)

pfba_solution = cobra.flux_analysis.pfba(model_copy)

#set the pfba_factor to 1.01 to find variability of pfba 
#i.e. this corresponds to 99% FVA
pfba_solution_101 = flux_variability_analysis(model_copy,pfba_factor = 1.01)

#make a dataframe & export
for index, row in flux_df.iterrows():  
    pfba = pfba_solution[row['model_id']]
    mini = pfba_solution_101.loc[row['model_id']]['minimum']
    maxi = pfba_solution_101.loc[row['model_id']]['maximum']
    flux_df.loc[index,'pfba'] = pfba
    flux_df.loc[index,'pfba_101_min'] = mini
    flux_df.loc[index,'pfba_101_max'] = maxi
    
flux_df.to_csv('../tables/Outputs/glcn_flux_pred_pfba.csv')